# 04 - Modelagem Preditiva
Autora: Fernanda Baptista de Siqueira  
Curso: MBA em Tecnologia para Negócios – AI, Data Science e Big Data  
Tema: Análise de Acidentes de Trânsito em Porto Alegre (2020–2024)  
Origem DataFrame: Equipe Armazém de Dados de Mobilidade - EAMOB/CIET  
https://dadosabertos.poa.br/dataset/acidentes-de-transito-acidentes (11/05/2025)  

### 1. Importa bibliotecas e funções. Carrega dados

In [1]:
from config import (
    pd, sns, plt, np,
    resumo_df, ajustar_tipos, 
    PATH_CLEAN, COLS_VEICULOS,
    COLS_CAT, COLS_INT, COLS_STR
)

# Pré-processamento e modelagem
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score

# Modelos
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor




### 2. Carrega Dados Tratados

In [2]:
# Carrega dataset já limpo
df = pd.read_parquet(PATH_CLEAN + 'df_limpo_chuva.parquet')
resumo_df(df)

# Define target (UPS = severidade)
y = df["ups"]
X = df.drop(columns=["ups"])

df.shape, X.shape, y.shape


Dimensões: (68837, 35)

Tipos de dados:
predial1                   Int32
queda_arr                  Int32
data              datetime64[ns]
feridos                    Int32
feridos_gr                 Int32
fatais                     Int32
auto                       Int32
taxi                       Int32
lotacao                    Int32
onibus_urb                 Int32
onibus_met                 Int32
onibus_int                 Int32
caminhao                   Int32
moto                       Int32
carroca                    Int32
bicicleta                  Int32
outro                      Int32
cont_vit                   Int32
ups                        Int32
patinete                   Int32
idacidente                 Int32
log1              string[python]
log2              string[python]
tipo_acid               category
dia_sem                 category
hora             timedelta64[ns]
noite_dia               category
regiao                  category
hora_int                   int64
dat

,predial1,queda_arr,data,feridos,feridos_gr,fatais,auto,taxi,lotacao,onibus_urb,onibus_met,onibus_int,caminhao,moto,carroca,bicicleta,outro,cont_vit,ups,patinete,idacidente,log1,log2,tipo_acid,dia_sem,hora,noite_dia,regiao,hora_int,data_hora,total_vitimas,soma_veiculos,data_meteo,chuva,chovendo
0,2500,0,2020-01-01,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,669196,AV FARRAPOS,AV SAO PEDRO,ABALROAMENTO,Quarta,0 days 02:20:00,NOITE,NORTE,2,2020-01-01 02:20:00,0,2,2020-01-01 02:00:00,0.00,0
1,598,0,2020-01-01,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,5,0,669089,AV BENTO GONCALVES,<NA>,ABALROAMENTO,Quarta,0 days 03:00:00,NOITE,LESTE,3,2020-01-01 03:00:00,1,2,2020-01-01 03:00:00,0.00,0
2,0,0,2020-01-01,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,669206,R SANTA FLORA,AV DA CAVALHADA,COLISÃO,Quarta,0 days 17:15:00,DIA,SUL,17,2020-01-01 17:15:00,0,2,2020-01-01 17:00:00,0.40,1
3,399,0,2020-01-01,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,669195,R SAO FRANCISCO DE ASSIS,<NA>,EVENTUAL,Quarta,0 days 17:15:00,DIA,NORTE,17,2020-01-01 17:15:00,0,1,2020-01-01 17:00:00,5.70,1
4,400,0,2020-01-01,1,1,0,0,0,0,0,0,0,0,1,0,1,0,1,5,0,683303,AV SENADOR TARSO DUTRA,<NA>,ABALROAMENTO,Quarta,0 days 23:00:00,NOITE,LESTE,23,2020-01-01 23:00:00,1,2,2020-01-01 23:00:00,0.00,0


((68837, 35), (68837, 34), (68837,))

### 3. Separa Treino, Validação e Teste

In [3]:
# 70% treino, 15% validação, 15% teste
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42
)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42
)

# Converte todos os pd.NA -> np.nan
for df_part in [X_train, X_valid, X_test]:
    df_part.replace({pd.NA: np.nan}, inplace=True)

# Converte colunas Int32 (nullable) para float64
for df_part in [X_train, X_valid, X_test]:
    int_cols = df_part.select_dtypes(include="Int32").columns
    if len(int_cols) > 0:
        df_part[int_cols] = df_part[int_cols].astype("float64")

X_train.shape, X_valid.shape, X_test.shape



((48185, 34), (10326, 34), (10326, 34))

### 4. Pré-processamento

In [4]:

# Categóricas: inclui object, category e string[python]
cat_cols = (
    X_train.select_dtypes(include=["object", "category", "string"]).columns.tolist()
)

# Datas
date_cols = X_train.select_dtypes(include=["datetime64[ns]", "datetimetz"]).columns.tolist()

num_cols = X_train.select_dtypes(
    include=["float32", "float64", "int32", "int64", "Int32", "Float32"]
).columns.difference(cat_cols + date_cols).tolist()

# Numéricas: vamos confiar na lista fixa (COLS_INT) + floats detectados
#num_cols = [c for c in COLS_INT if c in X_train.columns]
#num_cols += X_train.select_dtypes(include=["float32", "float64", "int32", "int64"]).columns.difference(cat_cols + date_cols).tolist()

cat_cols, num_cols, date_cols


(['log1', 'log2', 'tipo_acid', 'dia_sem', 'noite_dia', 'regiao'],
 ['auto',
  'bicicleta',
  'caminhao',
  'carroca',
  'chovendo',
  'chuva',
  'cont_vit',
  'fatais',
  'feridos',
  'feridos_gr',
  'hora_int',
  'idacidente',
  'lotacao',
  'moto',
  'onibus_int',
  'onibus_met',
  'onibus_urb',
  'outro',
  'patinete',
  'predial1',
  'queda_arr',
  'soma_veiculos',
  'taxi',
  'total_vitimas'],
 ['data', 'data_hora', 'data_meteo'])

### 5. Pré-processadores (linear vs árvores)
* Linear (Ridge) usa One-Hot (bom para linearidade)
* Árvores (RF/GB) usam Ordinal (rápido e não explode colunas)

In [5]:
# Pipelines básicos
numeric_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

def make_date_features(X_df: pd.DataFrame) -> pd.DataFrame:
    X_df = X_df.copy()
    out = pd.DataFrame(index=X_df.index)
    for c in X_df.columns:
        s = X_df[c]
        # se vier timezone-aware, normaliza para naive
        try:
            if hasattr(s.dt, "tz") and s.dt.tz is not None:
                s = s.dt.tz_convert("America/Sao_Paulo").dt.tz_localize(None)
        except Exception:
            # se não for datetime, retorna vazio (não deveria ocorrer aqui)
            return pd.DataFrame(index=X_df.index)
        out[f"{c}__year"]  = s.dt.year
        out[f"{c}__month"] = s.dt.month
        out[f"{c}__dow"]   = s.dt.dayofweek
        out[f"{c}__hour"]  = s.dt.hour
    return out

datetime_pipe = Pipeline([
    ("extract", FunctionTransformer(make_date_features, validate=False)),
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Categóricas:
# -> OHE sem 'sparse'/'sparse_output' para ser compatível com 1.1/1.2/1.4+
categorical_ohe_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

# Árvores: OrdinalEncoder (simples e rápido). 'encoded_missing_value' removido para compatibilidade ampla
categorical_ord_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ord", OrdinalEncoder(
        handle_unknown="use_encoded_value",
        unknown_value=-1
    ))
])


# Pré-processador para modelos lineares (OHE)
preprocess_linear = ColumnTransformer(
    transformers=[
        ("num",  numeric_pipe,   num_cols),
        ("date", datetime_pipe,  date_cols),
        ("cat",  categorical_ohe_pipe, cat_cols),
    ],
    sparse_threshold=0.3
)

# Pré-processador para árvores (Ordinal)
preprocess_trees = ColumnTransformer(
    transformers=[
        ("num",  numeric_pipe,   num_cols),
        ("date", datetime_pipe,  date_cols),
        ("cat",  categorical_ord_pipe, cat_cols),
    ],
    remainder="drop"
)

### 6. Modelos Candidatos

In [6]:
# Dicionário de modelos
modelos = {
    "Ridge (linear)": Pipeline([
        ("prep", preprocess_linear),
        ("model", Ridge(alpha=1.0))
    ]),
    "RandomForest": Pipeline([
        ("prep", preprocess_trees),
        ("model", RandomForestRegressor(
            n_estimators=300,
            max_depth=None,
            n_jobs=-1,
            random_state=42
        ))
    ]),
    "GradientBoosting": Pipeline([
        ("prep", preprocess_trees),
        ("model", GradientBoostingRegressor(
            n_estimators=300,
            learning_rate=0.05,
            random_state=42
        ))
    ])
}


In [9]:
print("Numéricas detectadas:", num_cols)
print("Categóricas detectadas:", cat_cols)
print("Datas detectadas:", date_cols)


Numéricas detectadas: ['auto', 'bicicleta', 'caminhao', 'carroca', 'chovendo', 'chuva', 'cont_vit', 'fatais', 'feridos', 'feridos_gr', 'hora_int', 'idacidente', 'lotacao', 'moto', 'onibus_int', 'onibus_met', 'onibus_urb', 'outro', 'patinete', 'predial1', 'queda_arr', 'soma_veiculos', 'taxi', 'total_vitimas']
Categóricas detectadas: ['log1', 'log2', 'tipo_acid', 'dia_sem', 'noite_dia', 'regiao']
Datas detectadas: ['data', 'data_hora', 'data_meteo']


### 7. Treino e Avaliação

In [7]:
# Avalia todos os modelos no conjunto de validação
resultados = {}

for nome, pipe in modelos.items():
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_valid)
    rmse = mean_squared_error(y_valid, y_pred, squared=False)
    r2 = r2_score(y_valid, y_pred)
    resultados[nome] = {"RMSE": rmse, "R²": r2}

# Mostra tabela ordenada pelo RMSE
pd.DataFrame(resultados).T.sort_values("RMSE")


TypeError: boolean value of NA is ambiguous

Escolher menor RMSE ou MAE e maior r² na validação.

In [ ]:
melhor_nome = ranking.index[0]
best_pipe = resultados[melhor_nome]["pipeline"]
print("🏆 Melhor modelo (val):", melhor_nome, resultados[melhor_nome]["val"])

# Avaliar no teste (se existir)
if not X_test.empty:
    yhat_test = best_pipe.predict(X_test)
    print("Teste:", avaliar(y_test, yhat_test))


7) Importância de features & interpretabilidade

Para árvores, usamos feature_importances_. Para modelos lineares, coeficientes.
Se você tiver SHAP instalado, incluí um bloco opcional.

In [ ]:
def nomes_features_transformados(preprocessor):
    # Recupera nomes após OneHotEncoder
    out = []
    # num
    out += num_cols
    # cat
    ohe = preprocessor.named_transformers_["cat"].named_steps["ohe"]
    cat_feat_names = ohe.get_feature_names_out(cat_cols).tolist()
    out += cat_feat_names
    return out

# Importância (quando aplicável)
try:
    model = best_pipe.named_steps["model"]
    feat_names = nomes_features_transformados(best_pipe.named_steps["prep"])

    if hasattr(model, "feature_importances_"):
        imp = pd.Series(model.feature_importances_, index=feat_names).sort_values(ascending=False).head(20)
        ax = imp[::-1].plot(kind="barh")
        ax.set_title(f"Top 20 importâncias — {melhor_nome}")
        plt.show()

    elif hasattr(model, "coef_"):
        coefs = pd.Series(model.coef_.ravel(), index=feat_names).sort_values()
        ax = coefs.head(10).plot(kind="barh"); plt.title(f"Coef. (neg) — {melhor_nome}"); plt.show()
        ax = coefs.tail(10).plot(kind="barh"); plt.title(f"Coef. (pos) — {melhor_nome}"); plt.show()
except Exception as e:
    print("Importância/coeficientes não disponíveis:", e)


SHAP (SHapley Additive exPlanations) é uma ferramenta poderosa para interpretar modelos de machine learning, especialmente modelos complexos como árvores de decisão e redes neurais. Ele atribui a cada feature uma contribuição para a previsão do modelo, permitindo entender como cada variável influencia o resultado.

In [ ]:
if HAS_SHAP and hasattr(best_pipe.named_steps["model"], "predict"):
    # Amostra para reduzir custo
    amostra = X_val.sample(min(3000, len(X_val)), random_state=42)
    X_val_trans = best_pipe.named_steps["prep"].transform(amostra)

    # Escolha do explainer depende do modelo
    try:
        explainer = shap.Explainer(best_pipe.named_steps["model"])
        shap_values = explainer(X_val_trans)
        shap.plots.beeswarm(shap_values, max_display=20)
    except Exception as e:
        print("SHAP não pôde rodar com este modelo:", e)
else:
    print("SHAP indisponível (biblioteca ausente) — pulando.")


8) Previsão temporal (mensal) com SARIMAX (opcional)

Previsão do total mensal de UPS para avaliar tendência de severidade.
Ajuste simples; melhore com covariáveis exógenas (chuva média mensal).

In [ ]:
if HAS_STATSMODELS:
    # Série mensal de UPS
    s_mensal = (df
                .dropna(subset=["data"])
                .set_index("data")
                .resample("M")["ups"].sum())

    # Treino até 2023, valida 2024, teste 2025-05
    s_train = s_mensal.loc[: "2023-12-31"]
    s_val   = s_mensal.loc["2024-01-01":"2024-12-31"]
    s_test  = s_mensal.loc["2025-01-01":"2025-05-31"]

    # Exógenas (chuva média mensal), se existir
    if "chuva" in df.columns:
        exo = df.set_index("data").resample("M")["chuva"].mean()
        exo_train = exo.loc[s_train.index]
        exo_val   = exo.loc[s_val.index]
        exo_test  = exo.loc[s_test.index] if not s_test.empty else None
    else:
        exo_train = exo_val = exo_test = None

    # Modelo SARIMAX simples
    mod = sm.tsa.statespace.SARIMAX(
        s_train, order=(1,1,1), seasonal_order=(1,1,1,12),
        exog=exo_train, enforce_stationarity=False, enforce_invertibility=False
    )
    res = mod.fit(disp=False)

    pred_val = res.get_forecast(steps=len(s_val), exog=exo_val)
    pred_mean_val = pred_val.predicted_mean
    ci_val = pred_val.conf_int()

    ax = s_mensal.plot(label="observado", alpha=0.6)
    pred_mean_val.plot(ax=ax, label="previsto (val)")
    ax.fill_between(ci_val.index, ci_val.iloc[:,0], ci_val.iloc[:,1], alpha=0.2)
    ax.axvspan(pd.Timestamp("2024-01-01"), pd.Timestamp("2024-12-31"), color="orange", alpha=0.1, label="val")
    if not s_test.empty:
        ax.axvspan(pd.Timestamp("2025-01-01"), pd.Timestamp("2025-05-31"), color="green", alpha=0.1, label="teste")
    ax.set_title("UPS mensal — observado vs previsão (SARIMAX)")
    ax.legend(); plt.show()

    # Erros em validação
    from math import sqrt
    rmse_val = sqrt(((s_val - pred_mean_val)**2).mean())
    mae_val = (s_val - pred_mean_val).abs().mean()
    print({"RMSE_val_mensal": rmse_val, "MAE_val_mensal": mae_val})
else:
    print("statsmodels indisponível — pulando SARIMAX.")


9) Conclusões e próximos passos (guia)

Desempenho: compare MAE/RMSE/R² entre os modelos; escolha o melhor (geralmente HistGBR e RF vão bem).

Interpretabilidade: use importâncias e (se possível) SHAP para entender sinais/efeitos.

Aprimoramentos:

Features: harmônicos de hora (seno/cosseno), sazonalidade (mês), feriados, interação chuva×noite_dia.

Espaço: agrupar log1 para reduzir cardinalidade (top-k + “outros”).

Validação: CV em blocos temporais (TimeSeriesSplit) além do hold-out por ano.

Incerteza: intervalos por quantile regression (HistGBR loss="quantile") para cenários pessimista/otimista.

Temporais: enriquecer SARIMAX com exógenas (chuva, feriados, mobilidade) e comparar com Prophet ou AutoARIMA (pmdarima).

---------

## Referenciais Teóricos

- Breiman (2001): *Two Cultures* → interpretação vs predição.
- Bishop (2006), Hastie, Tibshirani & Friedman (2009), Murphy (2012): fundamentos estatísticos e probabilísticos.
- Géron (2023), Müller & Guido (2016), Faceli et al. (2021): boas práticas em pipelines e scikit-learn.
- Zabala (2019, 2021): aplicações de modelagem preditiva.
- Pearl et al. (2016): inferência causal.
- Vilone & Longo (2020): interpretabilidade.
- Bao et al. (2020): incerteza espaço-temporal.
- Chen et al. (2025): ensembles avançados.